# Reactor agua: Acuña, Guadalupe y Molina, Nicolás TODO JUNTO

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import scipy.signal
from scipy.optimize import curve_fit
import scipy.stats as stats
import pandas as pd
import os
from  scipy.stats import chi2_contingency
#from statsmodels.stats.weightstats import DescrStatsW
import glob
from scipy.signal import butter, filtfilt
import scipy.integrate as spi
from scipy.stats import chi2

#ej de path=r'C:\Users\Sergio\Desktop\L6y7\18-04-23'
#path='C:/Users/Nicolás Molina/Desktop/L6-7/04-05-23-14kV'
#path=r'C:\Users\Sergio\Desktop\L6y7\04-05-23-19kV'

path=r'C:\Users\Sergio\Desktop\L6y7\12-05-23\e23456_agua\18kv'

#get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('matplotlib', 'qt5')


## arreglar csv

In [499]:
#ARREGLO LOS CSV
carpeta = path

# Iterar sobre cada archivo CSV en la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith('.csv'):
        ruta_archivo = os.path.join(carpeta, archivo)
        # Leer el contenido del archivo
        with open(ruta_archivo, 'r') as f:
            contenido = f.read()
        # Reemplazar las apariciones de ",,," por ",0,0,"
        contenido_modificado = contenido.replace(',,,', ',0,0,')
        # Escribir el contenido modificado en el archivo
        with open(ruta_archivo, 'w') as f:
            f.write(contenido_modificado)




## usando pasa-altos 

In [12]:
os.chdir (path)

files=glob.glob('*.csv')

files=files[0:1]  #SI QUIERO ANALIZAR SOLO UNOS POCOS O UNO

per0=0   #0 para primer periodo , 1000 para segundo

ampV=18000 #voltaje puesto en el generador aprox
ampI=2.5/1000
frecusada=8000  #aprox
 

#CON 1 SE PLOTEAN LAS COSAS CON 0 NO

graficoscrudos=1
graficoajuste=1
graficofiltro=1
graficoaplan=1
graficosfinal=1

if len(files) > 3:
    graficoscrudos=0
    graficoajuste=0
    graficofiltro=0
    graficoaplan=0
    graficosfinal=0

############################################
fig, (ax1, ax2) = plt.subplots(2, 1)
pers=[0,1000]   #0 para primer periodo , 1000 para segundo

plt.close("all")
potencias=[]
j=1
for file in files:
    med = np.loadtxt(file, delimiter=',', skiprows=16, unpack=True)
    t=med[3]
    Vdbd=med[4]
    Vstr=med[8]
    Istr=Vstr/50
    #print(str(j)+"-archivo="+file)

    if graficoscrudos==1:
        fig, (ax1, ax2) = plt.subplots(2, 1)
        # Graficar los datos en el primer subplot
        ax1.plot(t*1000, Vdbd/1000, color='blue', label='$V_{dbd}$')
        ax1.grid()
        ax1.set_title("Voltaje de alimentación")
        ax1.set_xlabel('tiempos [ms]')
        ax1.set_ylabel('V [kV]')
        ax1.legend()

        # Graficar los datos en el segundo subplot
        ax2.plot(t*1000, Istr*1000, color='red', label='$I_{str}$')
        ax2.grid()
        ax2.set_title("Corriente de streamers")
        ax2.set_xlabel('tiempo [ms]')
        ax2.set_ylabel('I [mA]')
        ax2.legend()

        fig.suptitle("archivo="+file)
    
        plt.show()

    def sin(x,T,a,b):
        y=a*np.sin(2*np.pi/T*x)+b
        return y

    init_vals=[1/frecusada,ampV,0]



    popt, pcov = curve_fit(sin, t, Vdbd,absolute_sigma=True,p0=init_vals)       #,p0=init_vals
    perr = np.sqrt(np.diag(pcov))

    valorT= popt[0]
    valora=popt[1]
    valorb=popt[2]
    err_T = perr[0]
    #print("periodo=",valorT)
    #print("frecuencia=",1/valorT)
    #print("len",len(t))
    #print("tiempo capturado en pantalla=",t[-1]-t[0])
    longper=valorT*10000000
    longper=round(longper)
    #print(longper)

    #print("valores inversa",valora1,valorb1)
    bins=np.linspace(t[0],t[-1],len(t))

    ajuste= sin(bins,valorT,valora,valorb)
    if graficoajuste==1:
        plt.figure()
        plt.grid()
        plt.title("Tensión de entrada"+file)
        plt.plot(t*1000,Vdbd/1000,label="datos crudos")
        plt.plot(bins*1000,ajuste/1000,label="ajuste seno")
        plt.xlabel("tiempo [ms]")
        plt.ylabel("V [kV]")
        plt.legend()

    frecuencia=1/valorT


    signal=Istr

    # Definir la frecuencia de corte del filtro
    cutoff = 100

    # Crear el filtro Butterworth de orden elevado
    order = 5
    nyquist = 0.5 * frecuencia
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high')

    # Aplicar el filtro a la señal
    filtered_signal = filtfilt(b, a, signal)

    Istr_plana=filtered_signal
    if graficofiltro==1:
        # Graficar la señal original y la señal filtrada
        plt.figure()
        plt.grid()
        plt.title("Aplanamiento de señal"+file)
        plt.plot(t*1000, signal*1000, label='Señal original')
        plt.plot(t*1000, filtered_signal*1000, label='Señal filtrada')
        plt.xlabel("tiempo [ms]")
        plt.ylabel("I [mA]")
        plt.legend()
        plt.show()

    p = 0.001 #valor umbral

    # Recorrer la lista y actualizar los valores menores a 'p' a cero
    for i in range(len(Istr_plana)):
        if Istr_plana[i] < p:
            Istr_plana[i] = 0
            
    if graficoaplan==1:
        plt.figure()
        plt.grid()
        plt.title("Aplanamiento total de señal"+file)
        plt.plot(t*1000, Istr_plana*1000, label='Señal filtrada')
        plt.xlabel("tiempo [ms]")
        plt.ylabel("I [mA]")
        plt.legend()
        plt.show()

    #os.chdir (r'C:\Users\Sergio\Desktop\labo5\difractiva\codigos\espectroscopia')
    #os.chdir (r'C:\Users\Nicolás Molina\Desktop\L6-7\13-4-23')
    #el len de los archivos es 2489
    # [500:500+longper]
    for per0 in pers:
        if graficosfinal==1:
            fig, (ax1, ax2) = plt.subplots(2, 1)


            # Graficar los datos en el primer subplot
            ax1.plot(t*1000, Vdbd/1000, color='grey', label='$V_{dbd}$')
            ax1.plot(t[per0:per0+longper]*1000, Vdbd[per0:per0+longper]/1000, color='blue', label='$V_{dbd}$ para P')
            ax1.grid()
            ax1.set_title("Voltaje de alimentación")
            ax1.set_xlabel('tiempos [ms]')
            ax1.set_ylabel('V [kV]')
            ax1.legend()

            # Graficar los datos en el segundo subplot
            ax2.plot(t*1000, Istr_plana*1000, color='grey', label='$I_{str}$')
            ax2.plot(t[per0:per0+longper]*1000, Istr_plana[per0:per0+longper]*1000, color='red', label='$I_{str}$ para P')
            ax2.grid()
            ax2.set_title("Corriente de streamers")
            ax2.set_xlabel('tiempo [ms]')
            ax2.set_ylabel('I [mA]')
            ax2.legend()

            fig.suptitle("archivo="+file)

            plt.show()
        Vpot=Vdbd[per0:per0+longper]
        Ipot=Istr_plana[per0:per0+longper] #paso la corriente a A
        N=len(Ipot)
        potencia=np.mean(Vpot*Ipot)
        potencias.append(potencia)
        #print("potencia=",potencia, "W")
        j=j+1
    

    
    
desviacion_estandar = stats.tstd(potencias)

print("--------FINAL--------")
if per0==1000:
    print("segundo periodo")
else:
    print("primer periodo")
#print("array potencias=",potencias)
print("potencia media de todo=",np.mean(potencias),"+-",desviacion_estandar,"W")
#print("desviaciòn estandar",desviacion_estandar)
print("maximo de potencias",max(potencias),"W","//numero de archivo",potencias.index(max(potencias))+1)
print("minimo de potencias",min(potencias),"W","//numero de archivo",potencias.index(min(potencias))+1)


--------FINAL--------
segundo periodo
potencia media de todo= 1.1094535658940334 +- 0.2190457222639123 W
maximo de potencias 1.2643422814967509 W //numero de archivo 2
minimo de potencias 0.9545648502913159 W //numero de archivo 1


## usando savgol

In [8]:
os.chdir (path)

files=glob.glob('*.csv')

n_archivo=3


#files=files[n_archivo-1:n_archivo]  #SI QUIERO ANALIZAR SOLO UNOS POCOS O UNO
   

ampV=18000 #voltaje puesto en el generador aprox
ampI=3/1000
frecusada=8000  #aprox
 

#CON 1 SE PLOTEAN LAS COSAS CON 0 NO

graficoscrudos=1
graficoajuste=0
graficofiltro=0
graficoaplan=0
graficosfinal=1

if len(files) > 1:
    graficoscrudos=0
    graficoajuste=0
    graficofiltro=0
    graficoaplan=0
    graficosfinal=0

############################################
fig, (ax1, ax2) = plt.subplots(2, 1)

plt.close("all")


pers=[0,1000]   #0 para primer periodo , 1000 para segundo

Vpp=[]
potencias=[]
corrientes=[]
j=1
for file in files:
    med = np.loadtxt(file, delimiter=',', skiprows=16, unpack=True)
    t=med[3]
    Vdbd=med[4]
    Vstr=med[8]
    Istr=Vstr/50
    #print(str(j)+"-archivo="+file)
    Vpp.append(max(Vdbd)-min(Vdbd))

    if graficoscrudos==1:
        fig, (ax1, ax2) = plt.subplots(2, 1)
        # Graficar los datos en el primer subplot
        ax1.plot(t*1000, Vdbd/1000, color='blue', label='$V_{dbd}$')
        ax1.grid()
        ax1.set_title("Voltaje de alimentación")
        ax1.set_xlabel('tiempos [ms]')
        ax1.set_ylabel('V [kV]')
        ax1.legend()

        # Graficar los datos en el segundo subplot
        ax2.plot(t*1000, Istr*1000, color='red', label='$I_{str}$')
        ax2.grid()
        ax2.set_title("Corriente de streamers")
        ax2.set_xlabel('tiempo [ms]')
        ax2.set_ylabel('I [mA]')
        ax2.legend()

        fig.suptitle("archivo="+file)
    
        plt.show()

    def sin(x,T,a,b):
        y=a*np.sin(2*np.pi/T*x)+b
        return y

    init_vals=[1/frecusada,ampV,0]



    popt, pcov = curve_fit(sin, t, Vdbd,absolute_sigma=True,p0=init_vals)       #,p0=init_vals
    perr = np.sqrt(np.diag(pcov))

    valorT= popt[0]
    valora=popt[1]
    valorb=popt[2]
    err_T = perr[0]


    popt, pcov = curve_fit(sin, t, Istr,absolute_sigma=True,p0=init_vals)       #,p0=init_vals
    perr = np.sqrt(np.diag(pcov))

    
    valorbIstr=popt[2]
    

    bins=np.linspace(t[0],t[-1],len(t))

    ajuste= sin(bins,valorT,valora,valorb)
    """
    if graficoajuste==1:
        plt.figure()
        plt.grid()
        plt.title("Tensión de entrada"+file)
        plt.plot(t*1000,Vdbd/1000,label="Datos crudos V")
        plt.plot(bins*1000,ajuste/1000,label="Ajuste seno")
        plt.xlabel("tiempo [ms]")
        plt.ylabel("V [kV]")
        plt.legend()
    
    frecuencia=1/valorT
    """

    signal=Istr
    from scipy.signal import savgol_filter

    window_length = 201
    polyorder = 3
    seno_filtrado = savgol_filter(Istr, window_length, polyorder)


    def sin2(x,T,a,b):
        y=a*np.sin(2*np.pi/T*x+b)+valorbIstr
        return y

    init_valsIstr=[1/frecusada,ampI,0]
    popt, pcov = curve_fit(sin2, t, seno_filtrado,absolute_sigma=True,p0=init_valsIstr)       #,p0=init_valsIstr
    perr = np.sqrt(np.diag(pcov))

    valorTf= popt[0]
    valoraf=popt[1]
    valorbf=popt[2]
    err_Tf = perr[0]

    Istr_ajuste=sin2(bins,valorTf,valoraf,valorbf)

    Istr_plana=Istr-Istr_ajuste

    longper=valorTf*10000000
    longper=round(longper)
    

    #Istr_plana=filtered_signal
    if graficofiltro==1:

        plt.figure()
        plt.title("savgol")
        plt.plot(t*1000, Istr*1000, label='Señal original')
        plt.plot(t*1000, seno_filtrado*1000, label='Señal filtrada')
        plt.xlabel('Tiempo [ms]')
        plt.ylabel('Amplitud(mA)')
        plt.grid(True)
        plt.legend(loc='best')
        plt.show()

        plt.figure()
        plt.grid()
        plt.title("Savgol + ajuste")
        plt.plot(t*1000,seno_filtrado*1000,label="Solo filtro")
        plt.plot(t*1000,Istr_ajuste*1000,label="Ajuste al filtro")
        plt.xlabel('Tiempo [ms]')
        plt.ylabel('Amplitud(mA)')
        plt.grid(True)
        plt.legend(loc='best')
        plt.show()

        # Graficar la señal original y la señal filtrada
        plt.figure()
        plt.grid()
        plt.title("Aplanamiento de señal"+file)
        plt.plot(t*1000, signal*1000, label='Señal original')
        plt.plot(t*1000, Istr_plana*1000, label='Señal filtrada')
        plt.xlabel("tiempo [ms]")
        plt.ylabel("I [mA]")
        plt.legend()
        plt.show()

    p = 0.001 #valor umbral

    # Recorrer la lista y actualizar los valores menores a 'p' a cero
    for i in range(len(Istr_plana)):
        if Istr_plana[i] < p:
            Istr_plana[i] = 0
            
    if graficoaplan==1:
        plt.figure()
        plt.grid()
        plt.title("Aplanamiento total de señal"+file)
        plt.plot(t*1000, Istr_plana*1000, label='Señal filtrada')
        plt.xlabel("tiempo [ms]")
        plt.ylabel("I [mA]")
        plt.legend()
        plt.show()


    for per0 in pers:
        if graficosfinal==1:
            fig, (ax1, ax2) = plt.subplots(2, 1)


            # Graficar los datos en el primer subplot
            ax1.plot(t*1000, Vdbd/1000, color='grey', label='$V_{dbd}$')
            ax1.plot(t[per0:per0+longper]*1000, Vdbd[per0:per0+longper]/1000, color='blue', label='$V_{dbd}$ para P')
            ax1.grid()
            ax1.set_title("Voltaje de alimentación")
            ax1.set_xlabel('tiempos [ms]')
            ax1.set_ylabel('V [kV]')
            ax1.legend()

            # Graficar los datos en el segundo subplot
            ax2.plot(t*1000, Istr_plana*1000, color='grey', label='$I_{str}$')
            ax2.plot(t[per0:per0+longper]*1000, Istr_plana[per0:per0+longper]*1000, color='red', label='$I_{str}$ para P')
            ax2.grid()
            ax2.set_title("Corriente de streamers")
            ax2.set_xlabel('tiempo [ms]')
            ax2.set_ylabel('I [mA]')
            ax2.legend()

            fig.suptitle("archivo="+file+"/periodo="+str(per0))

            plt.show()

        
        Vpot=Vdbd[per0:per0+longper]
        Ipot=Istr_plana[per0:per0+longper] #paso la corriente a A
        integralpot = spi.simps( Istr[:longper]*Vdbd[:longper],t[:longper])/valorTf 
        N=len(Ipot)
        potencia=np.mean(Vpot*Ipot)
        corriente=np.mean(Ipot)
    
        potencias.append(potencia)
        corrientes.append(corriente)
        #print("potencia=",potencia, "W")
        j=j+1
        

        
        
desviacion_estandar = stats.tstd(potencias)
    
def numarchmax(x):
    num=potencias.index(max(x))+1
    if num % 2 == 0:
        print("maximo en segundo periodo")
        return num/2 
    else:
        print("maximo en primer periodo")
        return (num-1)/2
    
def numarchmin(x):
    num=potencias.index(min(x))+1
    if num % 2 == 0:
        print("minimo en segundo periodo")
        return num/2
    else:
        print("minimo en primer periodo")
        return (num-1)/2
    
print(path)
if len(files)> 5:
    print("cantidad de archivos=",len(files))
    print("-------")


print("VOLTAJE Pk-Pk=",np.mean(Vpp))
print("POTENCIA MEDIA de todo=",np.mean(potencias),"+-",desviacion_estandar,"W")
print("-----")
print("MAXIMO de potencias",max(potencias),"W","//numero de archivo",round(numarchmax(potencias)))
print("MINIMO de potencias",min(potencias),"W","//numero de archivo",round(numarchmin(potencias)))
print("corriente media promedio=",np.mean(corrientes)*1000,"mA")
print("corriente media max=",max(corrientes)*1000,"mA")
print("corriente media min=",min(corrientes)*1000,"mA")




C:/Users/Nicolás Molina/Desktop/L6-7/e2-14kvprom64
POTENCIA MEDIA de todo= 0.0 +- 0.0 W
-----
maximo en primer periodo
MAXIMO de potencias 0.0 W //numero de archivo 0
minimo en primer periodo
MINIMO de potencias 0.0 W //numero de archivo 0
corriente media promedio= 0.0 mA
corriente media max= 0.0 mA
corriente media min= 0.0 mA


## pot

In [500]:

os.chdir (path)

files=glob.glob('*.csv')

n_archivo=1
plt.close("all")

#files=files[n_archivo-1:n_archivo]  #SI QUIERO ANALIZAR SOLO UNOS POCOS O UNO
   

ampV=19000 #voltaje puesto en el generador aprox
ampI=2/1000
frecusada=8000  #aprox
 

#CON 1 SE PLOTEAN LAS COSAS CON 0 NO

graficospot=0

if len(files) > 1:
    graficospot=0


per0=0
potencias=[]
corrientes=[]
Vpp=[]
for file in files:
    med = np.loadtxt(file, delimiter=',', skiprows=16, unpack=True)
    t=med[3]
    Vdbd=med[4]
    Vstr=med[8]
    Istr=Vstr/50
    Vpp.append(max(Vdbd)-min(Vdbd))

    def sin3(x,w,a,b,f):
        y=a*np.sin(2*np.pi*w*x+f)+b
        return y

    init_vals=[frecusada,ampI,0,0]



    popt, pcov = curve_fit(sin3, t, Istr,absolute_sigma=True,p0=init_vals)       #,p0=init_vals
    perr = np.sqrt(np.diag(pcov))

    valorb=popt[2]
    valorTf=1/popt[0]
    
    ajusteI=sin3(t,popt[0],popt[1],popt[2],popt[3])
    
    if len(files) <2:

        plt.figure()
        plt.plot(t,ajusteI)
        plt.plot(t,Istr)
        plt.plot(t,Istr-ajusteI)

        fig, (ax1, ax2) = plt.subplots(2, 1)


        # Graficar los datos en el primer subplot
        ax1.plot(t*1000, Vdbd/1000, color='grey', label='$V_{dbd}$')
        ax1.plot(t[per0:per0+longper]*1000, Vdbd[per0:per0+longper]/1000, color='blue', label='$V_{dbd}$ para P')
        ax1.grid()
        ax1.set_title("Voltaje de alimentación")
        ax1.set_xlabel('tiempos [ms]')
        ax1.set_ylabel('V [kV]')
        ax1.legend()

        # Graficar los datos en el segundo subplot
        ax2.plot(t*1000, Istr*1000, color='grey', label='$I_{str}$')
        ax2.plot(t[per0:per0+longper]*1000, Istr[per0:per0+longper]*1000, color='red', label='$I_{str}$ para P')
        ax2.grid()
        ax2.set_title("Corriente de streamers")
        ax2.set_xlabel('tiempo [ms]')
        ax2.set_ylabel('I [mA]')
        ax2.legend()

        plt.figure()
        plt.plot(t[:longper],(Istr[:longper]-np.mean(Istr[:2*longper]))*Vdbd[:longper])
        plt.grid()
        plt.title("a")
        plt.xlabel("tiempo [ms]")
        plt.ylabel("P [W]")
        plt.legend()
        plt.show()


    fig.suptitle("archivo="+file+"/periodo="+str(per0))

    plt.show()
    #np.mean(Istr[:2*longper])
    integralpot = spi.simps( (Istr[:longper]-valorb)*Vdbd[:longper],t[:longper])/valorTf  #spi.simps(y, x) Donde x es el array de valores en el eje x y y es el array de valores en el eje y. La función simps calcula la integral de y en función de x utilizando la regla de Simpson.
    integralI = spi.simps( (Istr[:longper]),t[:longper])/valorTf 
    potencias.append(integralpot)
    corrientes.append(integralI)
    
desviacion_estandarP = stats.tstd(potencias)
desviacion_estandarI = stats.tstd(corrientes)
print("VOLTAJE Pk-Pk=",np.mean(Vpp))
print("potencia media de todo=",np.mean(potencias),"+-",desviacion_estandarP,"W")
#print("maximo de potencias",max(potencias),"W","//numero de archivo",round(numarchmax(potencias)))
#print("minimo de potencias",min(potencias),"W","//numero de archivo",round(numarchmin(potencias)))
print("corriente media promedio=",np.mean(corrientes)*1000,"+-",desviacion_estandarI*1000,"mA")
#print("corriente media max=",max(corrientes)*1000,"mA")
#print("corriente media min=",min(corrientes)*1000,"mA")
#print("copiar y pegar esto=",np.mean(Vpp),",",np.mean(potencias),",",desviacion_estandarP,",",np.mean(corrientes)*1000,",",desviacion_estandarI*1000)
print(f"copiar y pegar esto={np.mean(Vpp)},{np.mean(potencias)},{desviacion_estandarP},{np.mean(corrientes)*1000},{desviacion_estandarI*1000}")


VOLTAJE Pk-Pk= 18200.0
potencia media de todo= 3.392076185792468 +- 0.08540042483689853 W
corriente media promedio= 0.4700678261909173 +- 0.015490941442380595 mA
copiar y pegar esto=18200.0,3.392076185792468,0.08540042483689853,0.4700678261909173,0.015490941442380595


# P e I vs Vpp
hecho para los electrodos 2 y 3

In [10]:
#os.chdir (r"C:\Users\Sergio\Desktop\L6y7")
os.chdir (r"C:\Users\Nicolás Molina\Desktop\L6-7\barridos 12-05-23")
med = np.loadtxt("barrido con 2.txt", delimiter=',', skiprows=1, unpack=True)

a=0   #datos 1
b=6

V=med[0]
P=med[1]
errP=med[2]
Istr=med[3]
errI=med[4]

V=V[a:b]
P=P[a:b]
Istr=Istr[a:b]
errP=errP[a:b]
errI=errI[a:b]
plt.close("all")


def lineal(x,a,b):
    y=a*x+b
    return y

popt, pcov = curve_fit(lineal, V/1000, P,sigma=errP,absolute_sigma=True)       #,p0=init_vals
perr = np.sqrt(np.diag(pcov))

valoraP=popt[0]
valorbP=popt[1]

ajusteP=lineal(V/1000,valoraP,valorbP)

popt, pcov = curve_fit(lineal, V/1000, Istr,sigma=errI,absolute_sigma=True)       #,p0=init_vals
perr = np.sqrt(np.diag(pcov))

valoraI=popt[0]
valorbI=popt[1]

ajusteI=lineal(V/1000,valoraI,valorbI)


fig, (ax1, ax2) = plt.subplots(2, 1)


# Graficar los datos en el primer subplot
ax1.scatter(V/1000, P, color='blue',marker="o",label="V vs P")
ax1.errorbar(V/1000, P,yerr=errP, color='blue',marker="o")
ax1.plot(V/1000,ajusteP,label="ajuste",c="red")
ax1.grid()
#ax1.set_title("V vs P",c="red")
ax1.set_xlabel('V [kV]')
ax1.set_ylabel('P [W]')
ax1.legend()

ax2.scatter(V/1000, Istr, color='grey',marker="o",label="V vs I")
ax2.errorbar(V/1000, Istr,yerr=errI, color='grey',marker="o")
ax2.plot(V/1000,ajusteI,label="ajuste",c="red")
ax2.grid()
#ax2.set_title("V vs I",c="red")
ax2.set_xlabel('V [kV]')
ax2.set_ylabel('I [mA]')
ax2.legend()

fig.suptitle("Barrido con 2")


def reduced_chi_square(observed, expected, degrees_of_freedom):
  
    chi_squared = np.sum((observed - expected)**2 / expected)
    reduced_chi_squared = chi_squared / degrees_of_freedom
    return reduced_chi_squared

    
chiP=reduced_chi_square(P,ajusteP,2)
chiI=reduced_chi_square(Istr,ajusteI,2)

print(chiP,"aaaaaaaaa",chiI)
print("paràmetros")
print("params I","pendiente=",valoraI,"/oo=",valorbI)
print("params P","pendiente=",valoraP,"/oo=",valorbP)

0.008719695373036419 aaaaaaaaa 0.00043049469402176233
paràmetros
params I pendiente= 0.02560053179964057 /oo= -0.2780932558674131
params P pendiente= 0.21717455240268113 /oo= -2.740036444949584


## en funcion de n para distintos voltajes


In [37]:
os.chdir (r"C:\Users\Nicolás Molina\Desktop\L6-7\12-05-23-pot en funcion de n con 5 E\P(n) e I(n)")
file="18kv cambiando n agua.txt"
med = np.loadtxt(file, delimiter=',', skiprows=1, unpack=True)

#a=0   #datos 1
#b=6

n=med[0]
P=med[1]
errP=med[2]
Istr=med[3]
errI=med[4]

V=V[a:b]
P=P[a:b]
#Istr=Istr[a:b]
#errP=errP[a:b]
#errI=errI[a:b]
plt.close("all")


def lineal(x,a,b):
    y=a*x+b
    return y

popt, pcov = curve_fit(lineal, n, P,sigma=errP,absolute_sigma=True)       #,p0=init_vals
perr = np.sqrt(np.diag(pcov))

valoraP=popt[0]
valorbP=popt[1]

ajusteP=lineal(n,valoraP,valorbP)

popt, pcov = curve_fit(lineal, n, Istr,sigma=errI,absolute_sigma=True)       #,p0=init_vals
perr = np.sqrt(np.diag(pcov))

valoraI=popt[0]
valorbI=popt[1]

ajusteI=lineal(n,valoraI,valorbI)


fig, (ax1, ax2) = plt.subplots(2, 1)


# Graficar los datos en el primer subplot
ax1.scatter(n, P, color='blue',marker="o")
ax1.errorbar(n, P,yerr=errP, color='blue',marker="o")
ax1.plot(n,ajusteP,label="ajuste",c="red")
ax1.grid()
ax1.set_title("n vs P",c="red")
ax1.set_xlabel('n')
ax1.set_ylabel('P [W]')
ax1.legend()

ax2.scatter(n, Istr, color='grey',marker="o")
ax2.errorbar(n, Istr,yerr=errI, color='grey',marker="o")
ax2.plot(n,ajusteI,label="ajuste",c="red")
ax2.grid()
ax2.set_title("n vs I",c="red")
ax2.set_xlabel('n')
ax2.set_ylabel('I [mA]')
ax2.legend()

fig.suptitle(file)


chiP=reduced_chi_square(P,ajusteP,2)
chiI=reduced_chi_square(Istr,ajusteI,2)

print(chiP,"aaaaaaaaa",chiI)
print("paràmetros")
print("params I","pendiente=",valoraI,"/oo=",valorbI)
print("params P","pendiente=",valoraP,"/oo=",valorbP)

0.021123904161540325 aaaaaaaaa 0.003967779225560721
paràmetros
params I pendiente= 0.10193797839085185 /oo= -0.015293438367917665
params P pendiente= 0.5288664331531252 /oo= 0.8667745137435958
